In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)

In [3]:
from sklearn.preprocessing import LabelEncoder

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
dt_suffix = '21_01_2023_20_08_24'

In [6]:
df_auctions = pd.read_csv(f'data/tabular/{dt_suffix}_auctions.csv')
df_itens = pd.read_csv(f'data/tabular/{dt_suffix}_itens.csv')
df_optionals = pd.read_csv(f'data/tabular/{dt_suffix}_optionals.csv')
df_bids = pd.read_csv(f'data/tabular/{dt_suffix}_bids.csv')

In [7]:
pd.set_option('display.max_columns', None)

In [8]:
df_itens.tail(3)

,auction_number,auction_item_lote,auction_item_type,auction_item_name,auction_item_url,auction_item_min_bid_value,auction_item_increment_value,auction_item_highest_bid,auction_item_date_text,auction_item_description,auction_item_model,auction_item_brand,auction_item_model_year,auction_item_market_price,auction_item_color,auction_item_gas_type,auction_item_km,auction_item_refurbished,execution_ts
11581,564,127,Presencial,NISSAN LIVINA XGEAR 18 2013/2014 PRATA,https://www.parquedosleiloes.com.br/leilao/564/lote/15838,NaN,NaN,NaN,NaN,NaN,LIVINA XGEAR 18,NISSAN,2013/2014,-,PRATA,FLEX,49505,Não,21/01/2023 20:08:24
11582,564,128,Presencial,GM - CHEVROLET PRISMA 2016/2016 BRANCA,https://www.parquedosleiloes.com.br/leilao/564/lote/15839,NaN,NaN,NaN,NaN,NaN,PRISMA,GM - CHEVROLET,2016/2016,-,BRANCA,FLEX,5363,Não,21/01/2023 20:08:24
11583,564,129,Presencial,CITROËN C3 90M TENDANCE 2013/2014 BRANCA,https://www.parquedosleiloes.com.br/leilao/564/lote/15840,NaN,NaN,NaN,NaN,NaN,C3 90M TENDANCE,CITROËN,2013/2014,-,BRANCA,FLEX,44530,Não,21/01/2023 20:08:24


In [9]:
 ## ajustando nomes de marcas
df_itens['auction_item_brand'] = df_itens['auction_item_brand'].apply(lambda x: x.strip())
df_itens['auction_item_brand'] = df_itens['auction_item_brand'].apply(lambda x: x[2:] if x[0:2] == 'I/' else x)
df_itens['auction_item_brand'] = df_itens['auction_item_brand'].apply(lambda x: x[4:] if x[0:4] == 'IMP/' else x)
df_itens['auction_item_brand'] = df_itens['auction_item_brand'].apply(lambda x: x[5:] if x[0:5] == 'IMP/ ' else x)

df_itens['auction_item_brand'].loc[df_itens['auction_item_brand'].str.contains("BENZ")] = 'MERCEDES BENZ'
df_itens['auction_item_brand'].loc[df_itens['auction_item_brand'].str.contains("GM")] = 'GM'
df_itens['auction_item_brand'].loc[df_itens['auction_item_brand'].str.contains("CHEV")] = 'GM'
df_itens['auction_item_brand'].loc[df_itens['auction_item_brand'].str.contains("HYUND")] = 'HYUNDAY'
df_itens['auction_item_brand'].loc[df_itens['auction_item_brand'].str.contains("VW")] = 'VW'
df_itens['auction_item_brand'].loc[df_itens['auction_item_brand'].str.contains("VOLKS")] = 'VW'
df_itens['auction_item_brand'].loc[df_itens['auction_item_brand'].str.contains("AUDI")] = 'AUDI'
df_itens['auction_item_brand'].loc[df_itens['auction_item_brand'].str.contains("CIT")] = 'CITROËN'
df_itens['auction_item_brand'].loc[df_itens['auction_item_brand'].str.contains("KIA")] = 'KIA'

c:\Users\gabri\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [10]:
## Retira marcas não identificadas ou que não são de veículos
df_itens = df_itens.loc[~ ( (df_itens.auction_item_brand.str.contains('HARLEY'))
                | (df_itens.auction_item_brand=='***')
                | (df_itens.auction_item_brand=='IVECO')
                | (df_itens.auction_item_brand=='DUCATI')
                | (df_itens.auction_item_brand.str.contains('ROYAL'))
                | (df_itens.auction_item_brand.str.contains('SHINERAY'))
                | (df_itens.auction_item_brand.str.contains('YAMAHA'))
            )]

In [11]:
def get_market_price(x):
    # print('x:',x)
    if x == '':
        return None
    else:
        try:
            return float(x.split(' ')[1].replace('.','').replace(',','.'))
        except:
            return None

In [12]:
def get_auction_item_date(x):
    # print(x)
    mp = {'jan':'01','fev':'02','mar':'03','abr':'04','mai':'05','jun':'06','jul':'07','ago':'08','set':'09','out':'10','nov':'11','dez':'12'}
    try:
        x = x.strip(' ')
        spl = x.split(' ')
        # print(spl)
        dt = spl[0].zfill(2) + '/' + mp[spl[2]] + '/' + spl[4]
        # print(dt)
        return dt
    except:
        return None

In [13]:
def get_km(x):
    try:
        return int(x)
    except:
        return None

In [14]:
df_itens['auction_item_market_price'] = df_itens['auction_item_market_price'].apply(get_market_price)

In [15]:
df_itens['auction_item_km'] = df_itens['auction_item_km'].apply(get_km)

In [16]:
df_itens['auction_date'] = pd.to_datetime(df_itens['auction_item_date_text'].apply(get_auction_item_date), format="%d/%m/%Y")
df_itens['perc_dec'] = (df_itens['auction_item_market_price'] - df_itens['auction_item_highest_bid'] ) / df_itens['auction_item_market_price'] # percentage of product sale decremet relative to market value

### Agregando dados de opcionais

In [17]:
## Existem opcionais duplicados para um mesmo índice
df_optionals = df_optionals.drop_duplicates(subset=['auction_number','auction_item_lote','auction_item_optionals'])

In [18]:
df_optionals.auction_item_optionals.unique()

array(['AIR BAG', 'ALARME', 'AR CONDICIONADO', 'CÂMBIO AUTOMÁTICO',
       'DIREÇÃO HIDRÁULICA', 'FREIOS ABS', 'VIDRO ELÉTRICO',
       'CÂMBIO MECÂNICO', 'TETO SOLAR', 'BLINDAGEM'], dtype=object)

In [19]:
df_optionals['value'] = 1

In [20]:
df_optionals = df_optionals.pivot(index=['auction_number','auction_item_lote'],columns='auction_item_optionals',values='value').reset_index().fillna(0)

In [21]:
df_optionals.head()

auction_item_optionals,auction_number,auction_item_lote,AIR BAG,ALARME,AR CONDICIONADO,BLINDAGEM,CÂMBIO AUTOMÁTICO,CÂMBIO MECÂNICO,DIREÇÃO HIDRÁULICA,FREIOS ABS,TETO SOLAR,VIDRO ELÉTRICO
0,564,1,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
1,564,2,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
2,564,3,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
3,564,4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,564,5,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0


In [22]:
df_join = df_itens.merge(df_optionals,on=['auction_number','auction_item_lote'],how='left')

In [23]:
df_join.head()

## Extrair modelo: Obter apenas primeiro nome da colua auction_item_model
## Obter keyword de auction_item_description: Avarias, sinistro e etc
## Obter idade do veículo no momento da audição. Usar auction_item_model_year e auction_date


,auction_number,auction_item_lote,auction_item_type,auction_item_name,auction_item_url,auction_item_min_bid_value,auction_item_increment_value,auction_item_highest_bid,auction_item_date_text,auction_item_description,auction_item_model,auction_item_brand,auction_item_model_year,auction_item_market_price,auction_item_color,auction_item_gas_type,auction_item_km,auction_item_refurbished,execution_ts,auction_date,perc_dec,AIR BAG,ALARME,AR CONDICIONADO,BLINDAGEM,CÂMBIO AUTOMÁTICO,CÂMBIO MECÂNICO,DIREÇÃO HIDRÁULICA,FREIOS ABS,TETO SOLAR,VIDRO ELÉTRICO
0,1081,1,Online,FIAT ARGO DRIVE 1.3 2019/2020 VERMELHA,https://www.parquedosleiloes.com.br/leilao/1081/lote/52961,39500.0,500.0,NaN,"Encerra em 25 jan 2023, 13:00:00","(SANTANDER) DOCUMENTO EM FASE DE EMISSÃO. PARA-BRISA NÃO ORIGINAL. PARA-CHOQUE DIANTEIRO E CAIXA DE AR LADO DIREITO AVARIADOS. LAUDO DO INMETRO E TODAS AS REGULARIZAÇÕES, SERÃO POR CONTA DO ARREMATANTE.",ARGO DRIVE 1.3,FIAT,2019/2020,66377.0,VERMELHA,FLEX,49208.0,Não,21/01/2023 20:08:24,NaT,NaN,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0
1,1081,2,Online,RENAULT SANDERO EXP1016V 2013/2014 VERMELHA,https://www.parquedosleiloes.com.br/leilao/1081/lote/52962,14500.0,500.0,NaN,"Encerra em 25 jan 2023, 13:01:00","(SANTANDER) DOCUMENTO CRLV EM NOSSO ESCRITÓRIO. PARA-CHOQUE DIANTEIRO, LATERAL TRASEIRA LADO DIREITO E CAIXA DE AR LADO DIREITO AVARIADOS. TODAS AS REGULARIZAÇÕES SERÃO POR CONTA DO ARREMATANTE.",SANDERO EXP1016V,RENAULT,2013/2014,32779.0,VERMELHA,FLEX,205523.0,Não,21/01/2023 20:08:24,NaT,NaN,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
2,1081,3,Online,CITROEN C4CACTUS FEEL AT 2021/2022 CINZA,https://www.parquedosleiloes.com.br/leilao/1081/lote/52963,64500.0,500.0,NaN,"Encerra em 25 jan 2023, 13:02:00",DOCUMENTO CRLV EM NOSSO ESCRITÓRIO.,C4CACTUS FEEL AT,CITROËN,2021/2022,99120.0,CINZA,FLEX,50670.0,Não,21/01/2023 20:08:24,NaT,NaN,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0
3,1081,4,Online,CHEVROLET ONIX 1.0MT LT 2015/2015 BRANCA,https://www.parquedosleiloes.com.br/leilao/1081/lote/52964,20500.0,500.0,NaN,"Encerra em 25 jan 2023, 13:03:00","(SANTANDER) DOCUMENTO CRLV EM NOSSO ESCRITÓRIO. PARA-CHOQUES, CAIXA DE AR LADO DIREITO E PORTAS LADO DIREITO AVARIADOS. TODAS AS REGULARIZAÇÕES SERÃO POR CONTA DO ARREMATANTE.",ONIX 1.0MT LT,GM,2015/2015,46181.0,BRANCA,FLEX,95584.0,Não,21/01/2023 20:08:24,NaT,NaN,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
4,1081,5,Online,I/HYUNDAI VELOSTER 2011/2012 PRETA,https://www.parquedosleiloes.com.br/leilao/1081/lote/52965,27500.0,500.0,NaN,"Encerra em 25 jan 2023, 13:04:00","(SANTANDER) DOCUMENTO EM FASE DE EMISSÃO. PARA-LAMA TRASEIRO ESQUERDO, LATERAL TRASEIRA LADO DIREITO E PAINEL TRASEIRO AVARIADOS. TODAS AS REGULARIZAÇÕES, SERÃO POR CONTA DO ARREMATANTE.",VELOSTER,HYUNDAY,2011/2012,61239.0,PRETA,GASOLINA,173134.0,Não,21/01/2023 20:08:24,NaT,NaN,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0


In [24]:
## Extrair modelo: Obter apenas primeiro nome da colua auction_item_model
df_join['model_name'] = df_join.auction_item_model.apply(lambda x: x.split(' ')[0] )

In [25]:
pd.options.display.max_colwidth = 50

In [26]:
## Obter keyword de auction_item_description: Avarias, sinistro e etc
# df_join['auction_item_description'].iloc[150:200].head(50)
df_join['SEM_CHAVE'] = df_join['auction_item_description'].str.contains('SEM CHAVE', case=False, na = False).replace(False,0).replace(True,1)
df_join['TRINCADO'] = df_join['auction_item_description'].str.contains('trincado', case=False, na = False).replace(False,0).replace(True,1)
df_join['AVARIADO'] = df_join['auction_item_description'].str.contains('AVARIADO', case=False, na = False).replace(False,0).replace(True,1)
df_join['SINISTRO'] = df_join['auction_item_description'].str.contains('SINISTRO', case=False, na = False).replace(False,0).replace(True,1)
df_join['RECALL'] = df_join['auction_item_description'].str.contains('RECALL', case=False, na = False).replace(False,0).replace(True,1)
df_join['ROUBO'] = df_join['auction_item_description'].str.contains('ROUBO', case=False, na = False).replace(False,0).replace(True,1)

In [27]:
df_join.auction_date.isna().sum()

1084

In [28]:
# Obtém idade do lote na data do leilão
def obter_idade(x):
    # print(x)
    try:
        model_year = x.auction_item_model_year
        if '/' in model_year:
            inicial_year = float(model_year.split('/')[0][-2:]) + 0.5
        else:
            inicial_year = float(model_year[-2:])
        
        auction_year = x.auction_date
        final_year = float(str(auction_year.year)[-2:])
        final_year = final_year + auction_year.month/12

        return final_year - inicial_year
    except:
        return np.nan
    

In [29]:
## Obter idade do veículo no momento do leilão. Usar auction_item_model_year e auction_date
df_join['item_age'] = df_join[['auction_item_model_year','auction_date']].apply(obter_idade,axis=1)

In [30]:
df_join.head(3)

,auction_number,auction_item_lote,auction_item_type,auction_item_name,auction_item_url,auction_item_min_bid_value,auction_item_increment_value,auction_item_highest_bid,auction_item_date_text,auction_item_description,auction_item_model,auction_item_brand,auction_item_model_year,auction_item_market_price,auction_item_color,auction_item_gas_type,auction_item_km,auction_item_refurbished,execution_ts,auction_date,perc_dec,AIR BAG,ALARME,AR CONDICIONADO,BLINDAGEM,CÂMBIO AUTOMÁTICO,CÂMBIO MECÂNICO,DIREÇÃO HIDRÁULICA,FREIOS ABS,TETO SOLAR,VIDRO ELÉTRICO,model_name,SEM_CHAVE,TRINCADO,AVARIADO,SINISTRO,RECALL,ROUBO,item_age
0,1081,1,Online,FIAT ARGO DRIVE 1.3 2019/2020 VERMELHA,https://www.parquedosleiloes.com.br/leilao/108...,39500.0,500.0,NaN,"Encerra em 25 jan 2023, 13:00:00",(SANTANDER) DOCUMENTO EM FASE DE EMISSÃO. PARA...,ARGO DRIVE 1.3,FIAT,2019/2020,66377.0,VERMELHA,FLEX,49208.0,Não,21/01/2023 20:08:24,NaT,NaN,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,ARGO,0.0,0.0,1.0,0.0,0.0,0.0,NaN
1,1081,2,Online,RENAULT SANDERO EXP1016V 2013/2014 VERMELHA,https://www.parquedosleiloes.com.br/leilao/108...,14500.0,500.0,NaN,"Encerra em 25 jan 2023, 13:01:00",(SANTANDER) DOCUMENTO CRLV EM NOSSO ESCRITÓRIO...,SANDERO EXP1016V,RENAULT,2013/2014,32779.0,VERMELHA,FLEX,205523.0,Não,21/01/2023 20:08:24,NaT,NaN,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,SANDERO,0.0,0.0,1.0,0.0,0.0,0.0,NaN
2,1081,3,Online,CITROEN C4CACTUS FEEL AT 2021/2022 CINZA,https://www.parquedosleiloes.com.br/leilao/108...,64500.0,500.0,NaN,"Encerra em 25 jan 2023, 13:02:00",DOCUMENTO CRLV EM NOSSO ESCRITÓRIO.,C4CACTUS FEEL AT,CITROËN,2021/2022,99120.0,CINZA,FLEX,50670.0,Não,21/01/2023 20:08:24,NaT,NaN,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,C4CACTUS,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [31]:
df = df_join[['auction_number', 'auction_item_lote', 'auction_item_type',
       'auction_item_min_bid_value',   'auction_item_highest_bid',
       'auction_item_brand', 'auction_item_market_price', 'auction_item_color',
       'auction_item_gas_type', 'auction_item_km', 'auction_date',
       'AIR BAG', 'ALARME', 'AR CONDICIONADO',
       'BLINDAGEM', 'CÂMBIO AUTOMÁTICO', 'CÂMBIO MECÂNICO',
       'DIREÇÃO HIDRÁULICA', 'FREIOS ABS', 'TETO SOLAR', 'VIDRO ELÉTRICO',
       'model_name', 'SEM_CHAVE', 'TRINCADO', 'AVARIADO', 'SINISTRO', 'RECALL',
       'ROUBO', 'item_age']]
       # auction_item_km

In [32]:
df.head()

,auction_number,auction_item_lote,auction_item_type,auction_item_min_bid_value,auction_item_highest_bid,auction_item_brand,auction_item_market_price,auction_item_color,auction_item_gas_type,auction_item_km,auction_date,AIR BAG,ALARME,AR CONDICIONADO,BLINDAGEM,CÂMBIO AUTOMÁTICO,CÂMBIO MECÂNICO,DIREÇÃO HIDRÁULICA,FREIOS ABS,TETO SOLAR,VIDRO ELÉTRICO,model_name,SEM_CHAVE,TRINCADO,AVARIADO,SINISTRO,RECALL,ROUBO,item_age
0,1081,1,Online,39500.0,NaN,FIAT,66377.0,VERMELHA,FLEX,49208.0,NaT,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,ARGO,0.0,0.0,1.0,0.0,0.0,0.0,NaN
1,1081,2,Online,14500.0,NaN,RENAULT,32779.0,VERMELHA,FLEX,205523.0,NaT,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,SANDERO,0.0,0.0,1.0,0.0,0.0,0.0,NaN
2,1081,3,Online,64500.0,NaN,CITROËN,99120.0,CINZA,FLEX,50670.0,NaT,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,C4CACTUS,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,1081,4,Online,20500.0,NaN,GM,46181.0,BRANCA,FLEX,95584.0,NaT,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,ONIX,0.0,0.0,1.0,0.0,0.0,0.0,NaN
4,1081,5,Online,27500.0,NaN,HYUNDAY,61239.0,PRETA,GASOLINA,173134.0,NaT,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,VELOSTER,0.0,0.0,1.0,0.0,0.0,0.0,NaN


In [33]:
for i in df.columns[2:]:
    print('*'*100)
    print(i)
    print('Nulls:', df[i].isna().sum())
    print(df[i].value_counts().head(5))

****************************************************************************************************
auction_item_type
Nulls: 0
Online                 10644
Presencial               769
Presencial e Online       97
Name: auction_item_type, dtype: int64
****************************************************************************************************
auction_item_min_bid_value
Nulls: 782
9500.0     301
10000.0    271
8000.0     271
12000.0    268
11000.0    256
Name: auction_item_min_bid_value, dtype: int64
****************************************************************************************************
auction_item_highest_bid
Nulls: 2010
14000.0    206
15000.0    145
10000.0    137
16000.0    129
11000.0    129
Name: auction_item_highest_bid, dtype: int64
****************************************************************************************************
auction_item_brand
Nulls: 0
VW         2091
FIAT       1751
GM         1501
RENAULT    1014
FORD        963
Name: auction_item_

In [34]:
## auction_item_type Manter apenas quando tem termo 'Online' 
df = df.loc[df.auction_item_type.str.contains('Online', case=False, na = False)]

In [35]:
## auction_item_min_bid_value dropar nulos
df = df.loc[~df.auction_item_min_bid_value.isna()]

In [36]:
## auction_item_highest_bid dropar nulos
df = df.loc[~df.auction_item_highest_bid.isna()]

In [37]:
## RECALL dropar coluna
## ROUBO dropar coluna
df.drop(['RECALL','ROUBO'],axis=1,inplace=True)

In [38]:
df['target_percentage'] = 100 * (df['auction_item_highest_bid'] - df['auction_item_min_bid_value'] ) / df['auction_item_min_bid_value']

In [39]:
df['target_diff'] = df['auction_item_highest_bid'] - df['auction_item_min_bid_value']

In [40]:
df['perc_dec'] = 100 * (df['auction_item_market_price'] - df['auction_item_highest_bid'] ) / df['auction_item_market_price'] # percentage of product sale decremet relative to market value

In [41]:
df.tail()

,auction_number,auction_item_lote,auction_item_type,auction_item_min_bid_value,auction_item_highest_bid,auction_item_brand,auction_item_market_price,auction_item_color,auction_item_gas_type,auction_item_km,auction_date,AIR BAG,ALARME,AR CONDICIONADO,BLINDAGEM,CÂMBIO AUTOMÁTICO,CÂMBIO MECÂNICO,DIREÇÃO HIDRÁULICA,FREIOS ABS,TETO SOLAR,VIDRO ELÉTRICO,model_name,SEM_CHAVE,TRINCADO,AVARIADO,SINISTRO,item_age,target_percentage,target_diff,perc_dec
11231,573,61,Online,7000.0,7800.0,VW,NaN,PRETA,FLEX,NaN,2018-03-02,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,GOL,0.0,0.0,0.0,0.0,11.75,11.428571,800.0,NaN
11232,573,62,Online,8000.0,11200.0,RENAULT,NaN,PRATA,FLEX,NaN,2018-03-02,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,CLIO,0.0,0.0,0.0,0.0,8.75,40.000000,3200.0,NaN
11233,573,63,Online,7500.0,20100.0,VW,NaN,BRANCA,FLEX,NaN,2018-03-02,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,NOVO,0.0,0.0,0.0,0.0,4.75,168.000000,12600.0,NaN
11234,573,64,Online,34000.0,34400.0,MMC,NaN,PRATA,DIESEL,144111.0,2018-03-03,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,PAJERO,0.0,0.0,0.0,0.0,8.75,1.176471,400.0,NaN
11236,573,66,Online,22000.0,22000.0,LEXUS,NaN,PRETA,GASOLINA,86178.0,2018-03-05,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,RX300,0.0,0.0,0.0,0.0,17.75,0.000000,0.0,NaN


In [42]:
df.to_csv(f'data/treated/{dt_suffix}_pq_leiloes.csv',index=False,header=True)